# Import libraries


In [1]:
from math import ceil
import re
from datetime import datetime, timedelta
import time
import concurrent.futures

import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

### Parameters


In [2]:
pd.set_option('display.max_colwidth', None)

# short extraction parameters
num_adtn_collect = 3
collect_freq = 7
# Seclare today time to fix the time range
today = datetime.today()
# Declare API Key
API_KEY = '3397e8a6a37e89b03b08750a27575df8'

In [3]:
# 한국, 미국 데이터 9507개
channel_df = pd.read_csv('us_korea_channels.csv')
# channel_df = ch_df[81:]
channel_df = channel_df.drop_duplicates(subset=['channel_name', 'yt_fixed_url'], keep='last')

loaded_df = pd.read_csv("loaded_db.csv")
today = datetime.today().strftime('%Y-%m-%d')
try:
    loaded_df['check_date'] = loaded_df['check_date'].dt.date
    filter_dates = [today - timedelta(days=7), today - timedelta(days=14), today - timedelta(days=21)]
    # Filter the dataframe based on check_date
    loaded_df = loaded_df[loaded_df['check_date'].isin(filter_dates)]
except Exception:
    pass

In [4]:
channel_df.head()

,youtuberme_url,category,country,channel_name,subscribers,total_video_views,total_video_count,started,yt_url,yt_fixed_url,data_yn
0,https://us.youtubers.me//robert-breaker/youtuber-stats,education,United States,Robert Breaker,702000.0,99929372.0,1879.0,2009.0,https://us.youtubers.me//robert-breaker/youtube,https://www.youtube.com/channel/UCPkTFG8FeBL6iR8YemTaMYQ/shorts,y
1,https://us.youtubers.me//nick-jones-00fc3f40-721a-488f-b594-a78beed97744/youtuber-stats,education,United States,Nick Jones,760000.0,99644730.0,485.0,2018.0,https://us.youtubers.me//nick-jones-00fc3f40-721a-488f-b594-a78beed97744/youtube,https://www.youtube.com/channel/UCwfeFpvbkp8cgmJWaMO9K0g/shorts,y
2,https://us.youtubers.me//tactical-toolbox/youtuber-stats,education,United States,Tactical Toolbox,957000.0,99603977.0,431.0,2006.0,https://us.youtubers.me//tactical-toolbox/youtube,https://www.youtube.com/channel/UCM9IxdhUn4Sm8ITdod8Tm2Q/shorts,y
3,https://us.youtubers.me//steve-ram/youtuber-stats,education,United States,Steve Ram,500000.0,99339604.0,2202.0,2016.0,https://us.youtubers.me//steve-ram/youtube,https://www.youtube.com/channel/UC_9iG1I_ic5jUOyzR9vTMBg/shorts,y
4,https://us.youtubers.me//awesome-toys-tv/youtuber-stats,education,United States,So Sweet ASMR,907000.0,99023999.0,2619.0,2014.0,https://us.youtubers.me//awesome-toys-tv/youtube,https://www.youtube.com/channel/UCrZfx6rscDd5qOdi5ni8rtg/shorts,y


In [5]:
loaded_df.head()

,channel_name,shorts_url,shorts_title,shorts_description,shorts_thumbnail,shorts_view,shorts_likes,shorts_comments_num,shorts_comments_time,shorts_published_date,check_date
0,Robert Breaker,https://www.youtube.com/watch?v=qc0iqmlAR38,NaN,NaN,https://support.google.com/youtube/answer/3037019#zippy=%2Ccheck-that-youre-signed-into-youtube,"6,422",264.0,16.0,NaN,NaN,2024-10-28 00:00:00
1,Nick Jones,https://www.youtube.com/watch?v=fa_sSaHxTZ4,NaN,NaN,https://support.google.com/youtube/answer/3037019#zippy=%2Ccheck-that-youre-signed-into-youtube,"91,113",17000.0,553.0,NaN,NaN,2024-10-28 00:00:00
2,Steve Ram,https://www.youtube.com/watch?v=SqU7aeAPkrY,Will You Comply? New Coming Mandates in 2023,FULL VIDEO • TRUMP BLASTS Bringing Back COVID Mand... \n\nJoin this channel to get access to perks:\n / @steveram,https://rr4---sn-a5mlrnlz.googlevideo.com/videoplayback?expire=1730136361\u0026ei=yXQfZ66mII7p9fwP5buhyAk\u0026ip=88.80.138.153\u0026id=o-ADGRnyRXXWs0ey6VgIAwD6jjYgDnlAPCHxJTkzJkl3op\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1730114761%2C\u0026mh=aJ\u0026mm=31%2C26\u0026mn=sn-a5mlrnlz%2Csn-aigl6ned\u0026ms=au%2Conr\u0026mv=u\u0026mvi=4\u0026pl=24\u0026rms=au%2Cau\u0026bui=AQn3pFRkfuQgpTNQtV6Rx0EbF5Yc4uKr6aUfAnutSsW00_W-aS2OQMOFaE7gsVnw3NP65JrCNa4-KGwt\u0026spc=qtApAeYZ-ZHwiV-OQpLGoJu6pxBfbDgbCbfueRn9EF3Kj1O_YEXP-RXwuN3Q9Ko\u0026vprv=1\u0026svpuc=1\u0026mime=video%2Fmp4\u0026ns=O-70yoh2uoVVRBz9uXYov70Q\u0026rqh=1\u0026cnr=14\u0026ratebypass=yes\u0026dur=27.028\u0026lmt=1699161979496252\u0026mt=1730113623\u0026fvip=5\u0026fexp=51312688%2C51326931\u0026c=WEB\u0026sefc=1\u0026txp=6300224\u0026n=6ETZCBPEcrdbd7kj\u0026sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Ccnr%2Cratebypass%2Cdur%2Clmt\u0026sig=AJfQdSswRAIgHhpJ1qfecSaSqCus0fZqWNcrOzdyPRqlySu9RmchW08CIDjeqaCim2fsX1d2kyDcMjiaMzHJ5PMQVuYu9TmqqzhL\u0026lsparams=met%2Cmh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Crms\u0026lsig=ACJ0pHgwRgIhAONANxNBbu_SWBgC4wdGutJ5fFsXLMotTeJYmKk1wMJbAiEA98hcmjhkxEUyrMgrsEEBK2QLCYvssCkbuRNtM-x0rr0%3D,"56,703",3200.0,504.0,NaN,2023-09-04 15,2024-10-28 00:00:00
3,So Sweet ASMR,https://www.youtube.com/watch?v=k27UsRRjC5g,ASMR Marble run #marblerun #asmr,ASMR Marble run #marblerun #asmr,https://rr1---sn-vgqsknez.googlevideo.com/videoplayback?expire=1730136367\u0026ei=z3QfZ8qpLaveybgP4OWv8A8\u0026ip=71.4.186.115\u0026id=o-AGLlhTYSO80LtXyVwaPBGbJYxUwygCN8TM1d-DXioC0f\u0026itag=18\u0026source=youtube\u0026requiressl=yes\u0026xpc=EgVo2aDSNQ%3D%3D\u0026met=1730114767%2C\u0026mh=lN\u0026mm=31%2C26\u0026mn=sn-vgqsknez%2Csn-p5qddn7r\u0026ms=au%2Conr\u0026mv=u\u0026mvi=1\u0026pl=21\u0026rms=au%2Cau\u0026bui=AQn3pFS3Hz6UofnuTnhzhIHgQ2I4S2cXpFEYnxTsghTaDqOHyGlPyEFUXzQWm-gnWGYn0yfh-3jQ8NFh\u0026spc=qtApAY2nxRS4poXQExTgxfNVGGVxdhlokeTGd5KfjP5h48RoEKRk4QgGrRqqEGA\u0026vprv=1\u0026svpuc=1\u0026mime=video%2Fmp4\u0026ns=VOstOVf9Rc6ykhAlGFFx9oAQ\u0026rqh=1\u0026gir=yes\u0026clen=537287\u0026ratebypass=yes\u0026dur=15.168\u0026lmt=1720022156668305\u0026mt=1730113623\u0026fvip=4\u0026fexp=51264009%2C51312688%2C51326931\u0026c=WEB\u0026sefc=1\u0026txp=6300224\u0026n=WJ2bpJvqGE_qtbq2\u0026sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cxpc%2Cbui%2Cspc%2Cvprv%2Csvpuc%2Cmime%2Cns%2Crqh%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt\u0026sig=AJfQdSswRQIhAM6_54omB-ct_OuFjld8bTeU3YM9Sr9zV5A7dVNglygKAiBIoGr0ccLa9ggeBDU7yByufNgLXpuADZum_H9yGuUmeA%3D%3D\u0026lsparams=met%2Cmh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl%2Crms\u0026lsig=ACJ0pHgwRQIgZ7Os0lyhs_TXGgXsrEL9HKC-C7RlqvFUSHvRXge_TBcCIQCnjWVIdtaoKHs2V9opF3dVuns2pwCD6qF6V2Srt9l08w%3D%3D,"12,582",236.0,3.0,NaN,2024-07-03 08,2024-10-28 00:00:00
4,English with Jennifer,https://www.youtube.com/watch?v=7L3CTSU7MqU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-10-28 00:00:00


# 2.Functions


### 2_1. PreProcess


In [6]:
# CHANGE: Parse to string date 
def to_strdata(date):
    if isinstance(date, datetime):
        date = date.strftime('%Y-%m-%d')
    return date


# Updated function to include the <link rel="image_src" ...> part in the thumbnail extraction
def basic_shorts_info(text):
    # Shorts title
    match_title = re.search(r'<title>(.*?) - YouTube</title>', text)
    extracted_title = match_title.group(1) if match_title else np.nan

    # Shorts description
    description_pattern = r'attributedDescription":{"content":"(.*?)"'
    description_match = re.search(description_pattern, text)
    extracted_description = description_match.group(1) if description_match else None
    
    # Shorts total view (date and time)
    date_time_pattern = r'datePublished" content="(\d{4}-\d{2}-\d{2})T(\d{2}):'
    match_date = re.search(date_time_pattern, text)
    extracted_published = f"{match_date.group(1)} {match_date.group(2)}" if match_date else np.nan

    # Thumbnail (updated pattern to include <link rel="image_src" ...)
    thumbnail_pattern = r'<link rel="image_src" href="(https?://i\.ytimg\.com/vi/[^/]+/maxres2\.jpg\?[^"]+)"'
    thumbnail_match = re.search(thumbnail_pattern, text)
    yt_shorts_thumbnail = thumbnail_match.group(1) if thumbnail_match else np.nan

    # Total view
    view_pattern = r'"viewCountEntityKey":"[^"]+","factoid":\{"factoidRenderer":\{"value":\{"simpleText":"([\d,]+)"'
    view_match = re.search(view_pattern, text)
    extracted_view = view_match.group(1) if view_match else np.nan

    # Total likes
    like_pattern = r'"iconName":"LIKE","title":"([\d.]+)([KM]?)"'
    like_match = re.search(like_pattern, text)
    if like_match:
        extracted_likes = float(like_match.group(1))
        suffix = like_match.group(2)
        if suffix == 'K':
            extracted_likes *= 1000
        elif suffix == 'M':
            extracted_likes *= 1000000
    else:
        extracted_likes = np.nan

    # Total comments    
    comment_num_pattern = r'"text":"Comments"}\]},"contextualInfo":\{"runs":\[\{"text":"([\d.]+)([KM]?)"'
    comment_num_match = re.search(comment_num_pattern, text)
    if comment_num_match:
        extracted_comments_number = float(comment_num_match.group(1))
        suffix = comment_num_match.group(2)
        if suffix == 'K':
            extracted_comments_number *= 1000
        elif suffix == 'M':
            extracted_comments_number *= 1000000
    else:
        extracted_comments_number = np.nan

    # YT shorts comment results and comment creation time
    comments_time_text_pattern = r'<span[^>]*id="published-time-text"[^>]*>\s*<a[^>]*>\s*(.*?)\s*<\/a>.*?<yt-attributed-string[^>]*>\s*<span[^>]*class="yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap"[^>]*>\s*(.*?)\s*<\/span>'
    
    comments_time_matches = re.findall(comments_time_text_pattern, text, re.DOTALL)
    
    # Process and clean comments
    extracted_comments_time_matches = []
    if comments_time_matches:
        for time, comment in comments_time_matches:
            soup = BeautifulSoup(comment, 'html.parser')
            cleaned_comment = soup.get_text()
            extracted_comments_time_matches.append((time.strip(), cleaned_comment.strip()))
    else:
        extracted_comments_time_matches = np.nan

    return (extracted_title, extracted_description, extracted_published, yt_shorts_thumbnail, 
            extracted_view, extracted_likes, extracted_comments_number, extracted_comments_time_matches)


def convert_subscribers(subscriber_text):
    """Convert subscriber text to a numeric value."""
    if 'K' in subscriber_text:
        return int(float(subscriber_text.replace('K', '').strip()) * 1000)
    elif 'M' in subscriber_text:
        return int(float(subscriber_text.replace('M', '').strip()) * 1000000)
    return int(subscriber_text.strip())

def extract_total_video_count(video_text):
    """Extract total video count as an integer from the input text."""
    # Check for formats like '3.1K' and convert accordingly
    if 'K' in video_text:
        return int(float(video_text.replace('K', '').strip()) * 1000)
    elif 'M' in video_text:
        return int(float(video_text.replace('M', '').strip()) * 1000000)
    number_pattern = r'(\d+) videos'
    match = re.search(number_pattern, video_text)
    return int(match.group(1)) if match else None  # Return the number part if found

def extract_description(description_text):
    """Extract description from the HTML content."""
    lines = description_text.splitlines()
    first_line = lines[0].strip() if len(lines) > 0 else None
    second_line = lines[1].strip() if len(lines) > 1 else None
    return first_line, second_line  # Get the first two lines
#################################################################3


### 2_2. Scraping


In [7]:
######################## Scraping code #################################

# 페이지 스크래핑 함수
def scrape_page(url):
    scraper_api_url = 'https://api.scraperapi.com'
    params = {
        'api_key': API_KEY,
        'url': url,
    }
    try:
        response = requests.get(scraper_api_url, params=params)
        if response.status_code == 200:
            print(f"Successfully scraped: {url}")
            return {"url": url, "status": "success", "content": response.text}
        else:
            print(f"Failed to scrape {url}: Status {response.status_code}")
            return {"url": url, "status": "failed", "status_code": response.status_code}
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return {"url": url, "status": "error", "error": str(e)}

# 배치 스크래핑 함수
def scrape_batch(urls, requests_per_second):
    # list to save data
    results = []
    # perform concurrent calls
    with concurrent.futures.ThreadPoolExecutor(max_workers=requests_per_second) as executor:
        futures = []
        for url in urls:
            futures.append(executor.submit(scrape_page, url))
            time.sleep(1 / requests_per_second)  # 초당 요청 제한
        
        for future in concurrent.futures.as_completed(futures):
            results.append(future.result())
    return results

# 전체 URL을 배치로 나누고 스크래핑 실행
    # url_data : list of url
    
def scrape_all_urls(url_data, batch_size, requests_per_second):
    # 전체 URL 수
    total_urls_len = len(url_data)
    print(f'Total url length: {total_urls_len}')
    
    # number of batches for 101 urls, 50 batchs -> 3 
    num_batches = ceil(total_urls_len / batch_size)
    
    # save all results
    all_results = []
    
    # start batch for each iternation
    for batch_num in range(num_batches):
        
        # batch start index 0, 50, 100...
        start_idx = batch_num * batch_size
        
        # end index 
        # 50, 100 
        end_idx = min((batch_num + 1) * batch_size, total_urls_len)
        
        # get urls form the url data
        #batch_urls = url_data['youtuberme_url'].iloc[start_idx:end_idx].tolist()
        batch_urls = url_data[start_idx:end_idx].tolist()
        
        print(f"Scraping batch {batch_num + 1}/{num_batches}, URLs {start_idx + 1}-{end_idx}")
        
        # get batch results
        batch_results = scrape_batch(batch_urls, requests_per_second)
        all_results.extend(batch_results)
    
    return pd.DataFrame(all_results)

######################## End of scraping code #################################


def scanning_yt_shorts_page(api_key, channel_db, loaded_db):
    # List to store new shorts results in the format (channel_name, new_shorts_url)
    new_shorts_urls = []
    # List to store channels with no shorts_urls
    no_shorts_list = []
    
    # Initialize a list for the additional DataFrame
    additional_data = []

    # Iterate over each row in channel_db
    for index, row in channel_db.iterrows():
        # Only proceed if data_yn is "y"
        if row['data_yn'] != 'y':
            continue  # Skip the row if data_yn is not "y"
        
        channel_name = row['channel_name']
        yt_fixed_url = row['yt_fixed_url']
        print(channel_name)
        # Filter loaded_db where channel_name matches
        matching_rows = loaded_db[loaded_db['channel_name'] == channel_name]
        
        if not matching_rows.empty:
            # Sort by check_date to get the most recent row
            latest_row = matching_rows.sort_values(by='check_date', ascending=False).iloc[0]
            latest_url = latest_row['shorts_url']
        else:
            # If no match found in loaded_db, set latest_url to None to ensure data collection proceeds
            latest_url = None
        
        # Prepare the payload for ScraperAPI request, using yt_fixed_url
        payload = {
            'api_key': api_key,
            'url': yt_fixed_url
        }
        
        # Send the request to ScraperAPI
        r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)
        
        # Define the regex pattern to extract Shorts URLs
        shorts_url_pattern = r'webCommandMetadata":{"url":"(/shorts/\w+)"'
        # Find all matching Shorts URLs
        shorts_urls = re.findall(shorts_url_pattern, r.text)
        shorts_urls = ['https://www.youtube.com' + url.replace('/shorts/', '/watch?v=') for url in shorts_urls]

        print(f"shorts_urls for {channel_name}: ", shorts_urls)

        # Extract subscribers
        subscribers_pattern = r'(\d+K|\d+M|\d+) subscribers'
        subscribers_match = re.search(subscribers_pattern, r.text)
        last_subscribers = convert_subscribers(subscribers_match.group(1)) if subscribers_match else None
        
        # Extract total video count with the required prefix
        total_video_count_pattern = r'{"text":{"content":"([\d.]+[K|M]?) videos"'
        total_video_count_match = re.search(total_video_count_pattern, r.text)
        last_total_videos = extract_total_video_count(total_video_count_match.group(1)) if total_video_count_match else None
        
        # Extract description
        description_pattern = r'itemprop="description" content="(.*?)"'
        description_match = re.search(description_pattern, r.text, re.DOTALL)
        last_description = extract_description(description_match.group(1)) if description_match else (None, None)

        # Store the data for the DataFrame
        additional_data.append({
            'channel_name': channel_name,
            'yt_fixed_url': yt_fixed_url,
            'subscribers': last_subscribers,
            'total_video_count': last_total_videos,
            'description': last_description[0]  # Assuming we want the first line of the description
        })

        # Proceed based on shorts_urls availability
        if shorts_urls:
            if latest_url and latest_url in shorts_urls:
                # If there's a match, get all URLs before the latest one
                matching_index = shorts_urls.index(latest_url)
                for shorts_url in shorts_urls[:matching_index]:
                    new_shorts_urls.append((channel_name, shorts_url))
            else:
                # If no latest URL or no match in loaded_db, collect the first URL
                new_shorts_urls.append((channel_name, shorts_urls[0]))
        else:
            # If shorts_urls is empty, add the channel_name and yt_fixed_url to the no_shorts_list
            no_shorts_list.append((channel_name, yt_fixed_url))
    
    # Convert the no_shorts_list to a DataFrame
    no_shorts_df = pd.DataFrame(no_shorts_list, columns=['channel_name', 'yt_fixed_url'])
    
    # Create a DataFrame from the additional data
    df = pd.DataFrame(additional_data)

    return new_shorts_urls, no_shorts_df, df  # Return both the list, no_shorts DataFrame, and additional DataFrame


def update_channel_df_with_new_data(ch_df, new_data):
    """
    Update the channel DataFrame with new data based on matching channel names.
    
    Parameters:
    ch_df (pd.DataFrame): The original channel DataFrame to be updated.
    new_data (pd.DataFrame): The new data DataFrame containing updated information.
    
    Returns:
    pd.DataFrame: The updated channel DataFrame.
    """
    for index, new_row in new_data.iterrows():
        channel_name = new_row['channel_name']

        # Find matching channel in channel_df
        matching_row = ch_df[ch_df['channel_name'] == channel_name]

        if not matching_row.empty:
            # Update values if they're NaN or if the new value is not NaN
            ch_df.loc[ch_df['channel_name'] == channel_name, 'subscribers'] = (
                new_row['subscribers'] if pd.notna(new_row['subscribers']) else ch_df.loc[ch_df['channel_name'] == channel_name, 'subscribers']
            )
            
            ch_df.loc[ch_df['channel_name'] == channel_name, 'total_video_count'] = (
                new_row['total_video_count'] if pd.notna(new_row['total_video_count']) else ch_df.loc[ch_df['channel_name'] == channel_name, 'total_video_count']
            )
    
    return ch_df

def update_channel_df_with_new_shorts(df, new_shorts_urls_df):
    # Iterate over each row in new_shorts_urls_df to match with channel_df
    for index, row in new_shorts_urls_df.iterrows():
        target_channel_name = row['channel_name']
        
        # Update 'data_yn' in channel_df where 'channel_name' matches
        df.loc[df['channel_name'] == target_channel_name, 'data_yn'] = 'n'

    # Return the updated channel_df with all rows
    return df

# url 여러개 진행
def add_new_shorts_data(api_key, urls):
    data_list = []
    check_date = today
    # Separate channel names and shorts URLs
    channel_names = [item[0] for item in urls]
    shorts_urls = [item[1] for item in urls]
    
    for channel_name, url in zip(channel_names, shorts_urls):
        print(channel_name, url)
        shorts_url = url
        # Prepare payload and headers for the request
        payload = {
            'api_key': api_key,
            'url': url
            # ,'render': 'true'
        }
        # Make the GET request
        r = requests.get('https://api.scraperapi.com/', params=payload,  verify=False)
        # Extract the necessary data
        shorts_title, shorts_description, shorts_published, shorts_thumbnail, shorts_view, shorts_likes, shorts_comments_num, shorts_comments_time = basic_shorts_info(r.text)
        
        # Print the extracted information for debugging
        # print("title: ", channel_name)
        # print("shorts_title: ", shorts_title)
        # print("shorts_description: ", shorts_description)
        # print("shorts_published: ", shorts_published)
        # print("shorts_thumbnalil: ", shorts_thumbnail)
        # print("view: ", shorts_view)
        # print("likes: ", shorts_likes)
        # print("comments: ", shorts_comments_num)
        # print("shorts_comments_time: ", shorts_comments_time)

        # Append as a tuple to the data_list
        data_list.append((
            channel_name,
            shorts_url, 
            shorts_title,  
            shorts_description, 
            shorts_thumbnail, 
            shorts_view, 
            shorts_likes, 
            shorts_comments_num, 
            shorts_comments_time,
            shorts_published, 
            check_date

        ))
    
    return data_list

# 추가된 데이터 dataframe으로 loaed_db에 추가
def add_new_data_to_loaded_db(data_list, loaded_db):
    # Define the columns for the DataFrame
    columns = [
        'channel_name', 'shorts_url', 'shorts_title', 'shorts_description', 
        'shorts_thumbnail', 'shorts_view', 'shorts_likes', 
        'shorts_comments_num', 'shorts_comments_time', 
        'shorts_published_date', 'check_date'
    ]
    
    # Map the data_list to the correct columns and create a new DataFrame
    new_data_df = pd.DataFrame(data_list, columns=columns)
    
    # Append the new data to the loaded_db DataFrame
    updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)
    
    return updated_loaded_db



def collecting_past_shorts(api_key, loaded_df):    
    # Convert 'today' to a datetime object from the 'yyyy-mm-dd' format
    today_date = datetime.strptime(today, "%Y-%m-%d")
    #today_date = today.strftime("%Y-%m-%d")
    # Iterate over each row in loaded_df, checking check_date and shorts_url
    for idx, (check_dt, shorts_url) in enumerate(zip(loaded_df['check_date'], loaded_df['shorts_url'])):
        # Convert check_dt to datetime format for comparison
        check_dt_date = datetime.strptime(check_dt, "%Y-%m-%d") if isinstance(check_dt, str) else check_dt
        
        # Check if check_dt matches any of the past intervals (7, 14, or 21 days ago)
        if check_dt_date in [today_date - timedelta(days=days) for days in [7, 14, 21]]:
            payload = {
                'api_key': api_key,
                'url': shorts_url
            }
            
            # Send the request
            r = requests.get('https://api.scraperapi.com/', params=payload, verify=False)

            # Extract relevant information from the response
            yt_title, yt_shorts_description, yt_shorts_published, yt_shorts_thumbnail, yt_shorts_view, yt_shorts_likes, yt_shorts_comments_num, yt_shorts_comments_time = basic_shorts_info(r.text)
            
            # Prepare new row with updated values (keeping the same channel name and other values from loaded_df)
            new_row = {
                'channel_name': loaded_df.loc[idx, 'channel_name'], 
                'shorts_url': shorts_url, 
                'shorts_title': yt_title,
                'shorts_description': yt_shorts_description,
                'shorts_thumbnail': yt_shorts_thumbnail,
                'shorts_view': yt_shorts_view,
                'shorts_likes': yt_shorts_likes,
                'shorts_comments': yt_shorts_comments_num,
                'shorts_comments_time': yt_shorts_comments_time,
                'shorts_published_date': yt_shorts_published,
                'check_date': today  # Use today's date for the new check_date
            }
            
            # Append the new row to the DataFrame
            loaded_df = loaded_df.append(new_row, ignore_index=True)
        else:
            print("no data")
            pass

    return loaded_df

### Collect new shorts urls


In [8]:
def shorts_page_preprocessor(result):
            
    # Define the regex pattern to extract Shorts URLs
    shorts_url_pattern = r'webCommandMetadata":{"url":"(/shorts/\w+)"'
    # Find all matching Shorts URLs
    shorts_urls = re.findall(shorts_url_pattern, result)
    shorts_urls = ['https://www.youtube.com' + url.replace('/shorts/', '/watch?v=') for url in shorts_urls]


    # Extract subscribers
    subscribers_pattern = r'(\d+K|\d+M|\d+) subscribers'
    subscribers_match = re.search(subscribers_pattern, result)
    last_subscribers = convert_subscribers(subscribers_match.group(1)) if subscribers_match else None
            
    # Extract total video count with the required prefix
    total_video_count_pattern = r'{"text":{"content":"([\d.]+[K|M]?) videos"'
    total_video_count_match = re.search(total_video_count_pattern, result)
    last_total_videos = extract_total_video_count(total_video_count_match.group(1)) if total_video_count_match else None
            
    # Extract description
    description_pattern = r'itemprop="description" content="(.*?)"'
    description_match = re.search(description_pattern, result, re.DOTALL)
    last_description = extract_description(description_match.group(1)) if description_match else (None, None)
    
    return last_subscribers, last_total_videos, last_description, shorts_urls
        

def scanning_yt_shorts_page_cct(channel_df, loaded_df, batch_size, requests_per_second):
    # List to store new shorts results in the format (channel_name, new_shorts_url)
    new_shorts_urls = []
    # List to store channels with no shorts_urls
    no_shorts_list = []    
    # Initialize a list for the additional DataFrame
    additional_data = []
    
    target_df = channel_df[channel_df['data_yn'] == 'y']
    channel_names = target_df['channel_name']
    yt_fixed_urls = target_df['yt_fixed_url']
    latest_urls = []
    
    # Filter loaded_db where channel_name matches
    matching_df = loaded_df[loaded_df['channel_name'].isin(channel_names)]
    
    for channel_name in channel_names:
        matching_rows = matching_df[matching_df['channel_name'] == channel_name]
                
        if not matching_rows.empty:
            # Sort by check_date to get the most recent row
            latest_row = matching_rows.sort_values(by='check_date', ascending=False).iloc[0]
            latest_url = latest_row['shorts_url']
        else:
            # If no match found in loaded_db, set latest_url to None to ensure data collection proceeds
            latest_url = None
            
        latest_urls.append(latest_url)

    scrapped_data = scrape_all_urls(yt_fixed_urls, batch_size, requests_per_second)
    scrapped_contents = scrapped_data['content']
    
    last_subscribers_list = []
    last_total_videos_list = []
    last_description_list = []
    shorts_urls_list = []
    for scrapped_content in scrapped_contents:
        last_subscribers, last_total_videos, last_description, shorts_urls = shorts_page_preprocessor(scrapped_content)
        last_subscribers_list.append(last_subscribers)
        last_total_videos_list.append(last_total_videos)
        last_description_list.append(last_description[0])
        shorts_urls_list.append(shorts_urls)

    for shorts_urls, latest_url, yt_fixed_url, channel_name in zip(shorts_urls_list,latest_urls, yt_fixed_urls, channel_names):
        # Proceed based on shorts_urls availability
        if shorts_urls:
            if latest_url and latest_url in shorts_urls:
                # If there's a match, get all URLs before the latest one
                matching_index = shorts_urls.index(latest_url)
                for shorts_url in shorts_urls[:matching_index]:
                    new_shorts_urls.append((channel_name, shorts_url))
            else:
                # If no latest URL or no match in loaded_db, collect the first URL
                new_shorts_urls.append((channel_name, shorts_urls[0]))
        else:
            # If shorts_urls is empty, add the channel_name and yt_fixed_url to the no_shorts_list
            no_shorts_list.append((channel_name, yt_fixed_url))
    
    additional_data = {
        'channel_name': channel_names,
        'yt_fixed_url': yt_fixed_urls,
        'subscribers': last_subscribers_list,
        'total_video_count': last_total_videos_list,
        'description': last_description_list  # As
    }
    no_shorts_df = pd.DataFrame(no_shorts_list, columns=['channel_name', 'yt_fixed_url'])
    df = pd.DataFrame(additional_data)
    
    return new_shorts_urls, no_shorts_df, df  # Return both the list, no_shorts DataFrame, and additional DataFrame


# CHANGE: 50 concurrent call
new_shorts_urls, no_shorts_df, df = scanning_yt_shorts_page_cct(channel_df[:121], loaded_df, 50, 50)

Total url length: 112
Scraping batch 1/3, URLs 1-50
Successfully scraped: https://www.youtube.com/channel/UCWTFGPpNQ0Ms6afXhaWDiRw/shorts
Successfully scraped: https://www.youtube.com/channel/UCFIRm1Fv1VC4DZxmYyvNOTQ/shorts
Successfully scraped: https://www.youtube.com/channel/UCWhKFUSIbtGK1CANZjwTgsA/shorts
Successfully scraped: https://www.youtube.com/channel/UCrtd2wePvAl6RN_D-9jWVQQ/shorts
Successfully scraped: https://www.youtube.com/channel/UCwfeFpvbkp8cgmJWaMO9K0g/shortsSuccessfully scraped: https://www.youtube.com/channel/UCaapxaQKJFJ6XC56CHgeTzw/shorts

Successfully scraped: https://www.youtube.com/channel/UCL5Hf6_JIzb3HpiJQGqs8cQ/shorts
Successfully scraped: https://www.youtube.com/channel/UCB3jUEyCLRbCw7QED0vnXYg/shorts
Successfully scraped: https://www.youtube.com/channel/UChPVoBpjfJngjoKjxrwGSTw/shorts
Successfully scraped: https://www.youtube.com/channel/UCrZfx6rscDd5qOdi5ni8rtg/shorts
Successfully scraped: https://www.youtube.com/channel/UC08Ti1qPyHfHSjUnOHNlEaw/shorts
Su

In [9]:
new_shorts_urls[:5]

[('Robert Breaker', 'https://www.youtube.com/watch?v=g6lcT4EhNzM'),
 ('Nick Jones', 'https://www.youtube.com/watch?v=ZmaF4ezj1aY'),
 ('Steve Ram', 'https://www.youtube.com/watch?v=iIR4NFDDEeg'),
 ('So Sweet ASMR', 'https://www.youtube.com/watch?v=LqGq8OLIX1Y'),
 ('English with Jennifer', 'https://www.youtube.com/watch?v=fa_sSaHxTZ4')]

In [10]:
no_shorts_df.head()

,channel_name,yt_fixed_url
0,Tactical Toolbox,https://www.youtube.com/channel/UCM9IxdhUn4Sm8ITdod8Tm2Q/shorts
1,LinkedIn Learning,https://www.youtube.com/channel/UCikzJG7RbnNZhKLqqaXRM6A/shorts
2,Matt Granger,https://www.youtube.com/channel/UCL5Hf6_JIzb3HpiJQGqs8cQ/shorts
3,Ben Eater,https://www.youtube.com/channel/UCS0N5baNlQWJCUrhCEo8WlA/shorts
4,Spiritual Secrets,https://www.youtube.com/channel/UC0p8yCPxPjlPwJ81dGxCHoA/shorts


In [11]:
df.head()

,channel_name,yt_fixed_url,subscribers,total_video_count,description
0,Robert Breaker,https://www.youtube.com/channel/UCPkTFG8FeBL6iR8YemTaMYQ/shorts,961000.0,NaN,Now You See It is a media analysis channel that searches for meaning in unexpected places.
1,Nick Jones,https://www.youtube.com/channel/UCwfeFpvbkp8cgmJWaMO9K0g/shorts,8000000.0,NaN,Helping you make better videos and tell interesting stories.
2,Tactical Toolbox,https://www.youtube.com/channel/UCM9IxdhUn4Sm8ITdod8Tm2Q/shorts,780000.0,NaN,"Bringing you into the paracord world has been my objective since 2012. I enjoy tying knots and bracelets. Even more, I enjoy sharing my passion with others and I want to do it in a way that pleases all. My hope is that you take my instruction and create amazing paracord items and learn useful skills along the way."
3,Steve Ram,https://www.youtube.com/channel/UC_9iG1I_ic5jUOyzR9vTMBg/shorts,649000.0,1600.0,World&#39;s best and free medical lectures to prepare you for medical exams.
4,So Sweet ASMR,https://www.youtube.com/channel/UCrZfx6rscDd5qOdi5ni8rtg/shorts,9000000.0,NaN,Channel by Leonardo Pereznieto


### Collect additional channel information


In [12]:
updated_channel_df = update_channel_df_with_new_data(channel_df, new_shorts_urls[2])

AttributeError: 'tuple' object has no attribute 'iterrows'

### Update Channel


In [64]:
updated_channel_df = update_channel_df_with_new_shorts(channel_df, new_shorts_urls[1])
channel_df.update(updated_channel_df)
channel_df.to_excel("us_korea.xlsx", index=False)

### Collect new shorts data


In [65]:
data_list = add_new_shorts_data(API_KEY, new_shorts_urls[0])

Robert Breaker https://www.youtube.com/watch?v=rQBQ0LhYi9w


/Users/yunjaecho/Git/dnova_code/env/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Nick Jones https://www.youtube.com/watch?v=fa_sSaHxTZ4


/Users/yunjaecho/Git/dnova_code/env/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Steve Ram https://www.youtube.com/watch?v=SqU7aeAPkrY


...

/Users/yunjaecho/Git/dnova_code/env/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Drum Beats Online https://www.youtube.com/watch?v=5EQArCVZJEc


/Users/yunjaecho/Git/dnova_code/env/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


CNN 10 https://www.youtube.com/watch?v=giybdhzJdHg


/Users/yunjaecho/Git/dnova_code/env/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.scraperapi.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [66]:
loaded_df = add_new_data_to_loaded_db(data_list, loaded_df)

/var/folders/qg/b8v1nh_d69760npt2k4xd1jc0000gn/T/ipykernel_7613/3802664893.py:303: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_loaded_db = pd.concat([loaded_db, new_data_df], ignore_index=True)


### Collect past shorts


In [76]:
today_past_shorts = collecting_past_shorts(API_KEY, loaded_df)
loaded_df = pd.concat([loaded_df, today_past_shorts], ignore_index=True)
loaded_df = loaded_df.drop_duplicates()
loaded_df.to_excel("loaded_db.xlsx", index=False)

TypeError: strptime() argument 1 must be str, not datetime.datetime

In [81]:
today = datetime.today().strftime('%Y-%m-%d')

In [82]:
today

'2024-11-14'